In [1]:
from py2neo import Graph, Node, Relationship, NodeSelector
from datetime import datetime
import os

In [2]:
url = os.environ.get('GRAPHENEDB_URL', 'http://localhost:7474')
username = os.environ.get('NEO4J_USERNAME')
password = os.environ.get('NEO4J_PASSWORD')

In [3]:
graph = Graph(url + '/db/data/', username=username, password=password, bolt=False, auth=False)

In [4]:
class User:
    def __init__(self, user_id):
        self.user_id = user_id
        self.selector = NodeSelector(graph)
        self.graph = graph
        
    def find(self):
        user = self.selector.select("User", user_id=self.user_id)
        return list(user)
    
    def get_friends(self):
        query = '''
        MATCH (u:User {user_id: {id}})-[:FRIEND]->(f:Person)
        RETURN f
        '''
        friends = graph.run(query, user_id=self.user_id)
        return list(friends)
    
    def find_most_discussed_topic(self):
        query = '''
        MATCH(m:Message {user_id: {id}})
        RETURN distinct m.topic, count(*) as occurences ORDER BY occurences DESC
        '''
        topics = graph.run(query, id=self.user_id)
        return list(topics)
    
    def find_recent_topic(self):
        query = '''
        MATCH(m:Message {user_id: {id}})
        RETURN m.topic, m.last_query as last_query ORDER BY last_query DESC
        '''
        topics = graph.run(query, id=self.user_id)
        return list(topics)

    def find_message_by_sentiment(self):
        query = '''
        MATCH(m:Message {user_id: {id}})
        RETURN m, m.sentiment ORDER BY m.sentiment DESC
        '''
        messages = graph.run(query, id=0)
        return list(messages)
    
    def suggest_friends_by_interest(self):
        query = '''
        MATCH (p1:Person)-[:LIKES]->(hobby1)<-[:LIKES]-(p2:Person),
              (p3:Person)-[:LIKES]->(hobby1)
        WHERE p1.person_id = {id}
        AND NOT (p2)-[:FRIEND]->(p3)
        RETURN p2, p3
        '''
        friends = graph.run(query, id=0)
        return list(friends)
    
    def find_most_recent_friend(self):
        query = '''
        MATCH(u:User {user_id: {id}})-[]->(f:Person)
        RETURN f, f.first_mention as first_mention ORDER BY first_mention DESC
        '''
        friends = graph.run(query, id=0)
        return list(friends)
        
    def suggest_friends_by_location(self, city):
        query = '''
        MATCH (p1:Person), (p2:Person)
        WHERE p1.location = p2.location = {location}
        AND NOT (p1)-[:FRIEND]->(p2)
        AND p1.person_id <> p2.person_id
        RETURN p1, p2
        '''
        friends = graph.run(query, location=city)
        return list(friends)
    
    def find_events(self):
        query = '''
        MATCH (p1:Person)-[:ATTENDED]->(e:Event)
        WHERE NOT p1:User
        return p1, e
        '''
        events = graph.run(query, id=0)
        return list(events)

In [6]:
user = User(0)
user.find_events()

[('p1': (ee4e6c3:Person {age:22,first_mention:1520363851263,gender:"F",location:"Waterloo",name:"Emma",person_id:3,profession:"student"}), 'e': (e58b329:Event {action:"study",created_at:1520363860029,event_name:"school",last_query:1520363860029,location:"waterloo",name:"school",time:"0"})),
 ('p1': (fc3c6ef:Person {age:24,first_mention:1520363851263,gender:"M",location:"Waterloo",name:"Amine",person_id:5,profession:"student"}), 'e': (e58b329:Event {action:"study",created_at:1520363860029,event_name:"school",last_query:1520363860029,location:"waterloo",name:"school",time:"0"}))]

In [ ]:
user.find()[0]['location']

In [ ]:
def timestamp():
    epoch = datetime.utcfromtimestamp(0)
    now = datetime.now()
    delta = now - epoch
    return int(delta.total_seconds())

In [ ]:
query = '''
MATCH (n)
RETURN n;
'''
res = list(graph.run(query, id=0))
print(len(list(res)))

In [ ]:
query = '''
MATCH ()-->()
RETURN count(*);
'''
res = list(graph.run(query, id=0))
print(list(res))


In [ ]:
user = User(0)
user.find()

In [ ]:
user.find_most_recent_friend()[0]['f']['name']

In [ ]:
selector = NodeSelector(graph)
res = selector.select("User")

In [ ]:
user.find_most_recent_friend()

In [ ]:
topics

In [ ]:
u = User('Olivier')
result = u.find()
print(result[0])